In [1]:
import time
import requests

In [2]:
# Variables

_url = 'https://westus.api.cognitive.microsoft.com/emotion/v1.0/recognize'
_key = ''
_maxNumRetries = 10 

In [3]:
def processRequest( json, data, headers, params ):

    """
    Helper function to process the request to Project Oxford

    Parameters:
    json: Used when processing images from its URL. See API Documentation
    data: Used when processing image read from disk. See API Documentation
    headers: Used to pass the key information and the data type request
    """

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result 

In [4]:
# Load raw image file into memory
# pathToFileInDisk = list_of_images[0]
# with open( pathToFileInDisk, 'rb' ) as f:
#     data = f.read()

# headers = dict()
# headers['Ocp-Apim-Subscription-Key'] = '1479c1f069394b7688946ae820acb779'
# headers['Content-Type'] = 'application/octet-stream'

# json = None
# params = None

# result = processRequest( json, data, headers, params )
# print(result) 

In [24]:
import glob
list_of_images = glob.glob('originalPics/*.jpg')

In [25]:
# initialize microsoft api
headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/octet-stream'

json = None
params = None

In [29]:
# microsoft rate limit = 20 per minute
img_array = []
list_of_images = list_of_images[0:19]
len(list_of_images)

19

In [30]:
# create for loop for images here
for img in list_of_images:
    pathToFileInDisk = img
    with open( pathToFileInDisk, 'rb' ) as f:
        data = f.read()
    result = processRequest( json, data, headers, params )
    json_output = {'input': img.split('/')[1],
                   'output': result}
    img_array.append(json_output)

In [31]:
print(img_array)

[{'input': '1.jpg', 'output': [{'faceRectangle': {'height': 142, 'left': 89, 'top': 89, 'width': 142}, 'scores': {'anger': 0.225969955, 'contempt': 0.0185250733, 'disgust': 0.01193539, 'fear': 0.00115492323, 'happiness': 0.0022804304, 'neutral': 0.727707446, 'sadness': 0.00441818, 'surprise': 0.008008626}}]}, {'input': '10.jpg', 'output': [{'faceRectangle': {'height': 148, 'left': 70, 'top': 112, 'width': 148}, 'scores': {'anger': 1.98885682e-05, 'contempt': 0.0004118145, 'disgust': 0.000129953958, 'fear': 1.03576355e-07, 'happiness': 0.96344167, 'neutral': 0.03590715, 'sadness': 7.948746e-05, 'surprise': 9.915888e-06}}]}, {'input': '100.jpg', 'output': [{'faceRectangle': {'height': 101, 'left': 86, 'top': 166, 'width': 101}, 'scores': {'anger': 0.000235914078, 'contempt': 0.00340858754, 'disgust': 0.000318060134, 'fear': 2.24505538e-06, 'happiness': 0.5783524, 'neutral': 0.416641951, 'sadness': 0.00101243309, 'surprise': 2.839436e-05}}, {'faceRectangle': {'height': 58, 'left': 220, 't